In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymongo
from pymongo import MongoClient
from keras.utils import Sequence
from keras.utils.np_utils import to_categorical
from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [1]:
IP_ADDRESS = '3.17.145.221'
client = MongoClient(f"mongodb://cjm715:password@{IP_ADDRESS}/py2cpp")
db = client.py2cpp
print(db.python.estimated_document_count(), db.cpp.estimated_document_count())


py_df0 = pd.DataFrame(list(db.python.find()))
cpp_df0 = pd.DataFrame(list(db.cpp.find()))

py_df = py_df0.drop(columns = ['_id'])
cpp_df = cpp_df0.drop(columns = ['_id'])

py_df = py_df.drop_duplicates()
cpp_df = cpp_df.drop_duplicates()

py_df = py_df.reset_index()
cpp_df = cpp_df.reset_index()
py_df = py_df.drop(columns = ['index'])
cpp_df = cpp_df.drop(columns = ['index'])

py_df = py_df[py_df['language'] == 'Python 3']
cpp_df = cpp_df[cpp_df['language'] == 'C++']

py_chall_list = list(py_df['challenge_title'].unique())
cpp_chall_list = list(cpp_df['challenge_title'].unique())
chall_list = list(set(py_chall_list + cpp_chall_list))
py_df = py_df[py_df.challenge_title.isin(chall_list)]
cpp_df = cpp_df[cpp_df.challenge_title.isin(chall_list)]

# Only considers examples with ASCII characters
py_df = py_df[py_df.code.map(lambda s: len(s) == len(s.encode()))]
cpp_df = cpp_df[cpp_df.code.map(lambda s: len(s) == len(s.encode()))]

print(len(py_df),len(cpp_df))

NameError: name 'MongoClient' is not defined

In [38]:
py_df.head()

,challenge_difficulty,challenge_title,code,language,max_score,rank,score,success_rate
0,Easy,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",Python 3,10.0,1,10.0,0.9454
1,Easy,Simple Array Sum,"n = input()\nl = list(map(int, input().split('...",Python 3,10.0,1,10.0,0.9454
2,Easy,Simple Array Sum,import sys \n\nn = int(input())\ndata = sys.st...,Python 3,10.0,1,10.0,0.9454
3,Easy,Simple Array Sum,input()\nprint(sum([int(x) for x in input().sp...,Python 3,10.0,1,10.0,0.9454
4,Easy,Simple Array Sum,"N=int(input())\narray=map(int,input().split())...",Python 3,10.0,1,10.0,0.9454


In [39]:
cpp_df.head()

,challenge_difficulty,challenge_title,code,language,max_score,rank,score,success_rate
0,Easy,Simple Array Sum,#include <cmath>\n#include <cstdio>\n#include ...,C++,10.0,1,10.0,0.9454
1,Easy,Simple Array Sum,#include <cmath>\n#include <cstdio>\n#include ...,C++,10.0,1,10.0,0.9454
2,Easy,Simple Array Sum,#include <cmath>\n#include <cstdio>\n#include ...,C++,10.0,1,10.0,0.9454
3,Easy,Simple Array Sum,#include <cmath>\n#include <cstdio>\n#include ...,C++,10.0,1,10.0,0.9454
4,Easy,Simple Array Sum,#include <cmath>\n#include <cstdio>\n#include ...,C++,10.0,1,10.0,0.9454


In [40]:
py_df.to_pickle('py_df.pkl')
cpp_df.to_pickle('cpp_df.pkl')

In [35]:
cpp_df['challenge_title'].value_counts().iloc[0:]

train_df = pd.DataFrame()
challenge_title_list = []
python_code_list = []
cpp_code_list = []

for _, py_ex in tqdm(py_df.iterrows()):

    num_per_ex = 18
    temp_df = cpp_df[cpp_df['challenge_title']==py_ex.challenge_title]
    if len(temp_df)< num_per_ex:
        continue
        
    temp_df = temp_df.sample(n = num_per_ex)
    
    for i in range(num_per_ex):
        cpp_ex_code = temp_df['code'].iloc[i]

        challenge_title_list.append(py_ex.challenge_title)
        python_code_list.append(py_ex.code)
        cpp_code_list.append(cpp_ex_code)
    
        
df_as_dict = {'challenge_title' : challenge_title_list,
           'python_code' : python_code_list,
             'cpp_code' : cpp_code_list}    

train_df = pd.DataFrame.from_dict(df_as_dict)
train_df = train_df.drop_duplicates()
train_df

train_df.to_pickle('train_df.pkl')

27071 29828
11794 14617


In [36]:
print(len(train_df))

199151


In [41]:
unique_char = set()
for _,row in py_df.iterrows():
    for ch in row['code']:
        unique_char.add(ch)
print(unique_char)

{'\\', '7', 'w', 'T', '=', 'Y', 'W', 'b', 'K', 'F', '~', 'u', '>', 'M', 'o', 'e', 'U', ',', '8', '<', 'x', ']', '5', 'k', '&', '?', 'a', 'r', ':', 'L', '.', 'h', '1', '\n', ')', 'z', 'B', 'd', '}', ';', '{', '-', 'v', 'm', '(', 'H', 'C', 'j', 'n', 'q', 'D', 'A', '/', ' ', '$', 'I', '#', 'X', '`', 'Q', 'c', '!', 't', '"', 'Z', 's', 'p', '^', 'R', '|', '+', 'S', 'G', '0', "'", '*', 'P', 'g', 'E', 'i', 'y', '2', '4', 'f', 'V', '_', '[', 'O', 'J', '9', '@', '6', '%', 'N', 'l', '3'}


In [45]:
unique_char_2 = set()
for _,row in cpp_df.iterrows():
    for ch in row['code']:
        unique_char_2.add(ch)
print(unique_char_2)

{'\\', '7', 'w', 'T', '=', 'Y', 'W', 'b', 'K', 'F', '~', 'u', '>', 'M', 'o', 'e', 'U', ',', '8', '<', 'x', ']', '5', 'k', '&', '?', 'a', 'r', 'L', ':', '.', 'h', '1', '\n', ')', 'z', 'B', 'd', '}', ';', '{', '-', 'v', 'm', '(', 'H', 'C', 'j', 'n', 'q', 'D', 'A', '/', ' ', '$', 'I', '#', 'X', '`', 'Q', 'c', '!', 't', '"', 'Z', 's', 'p', '^', 'R', '|', '+', 'S', 'G', '0', "'", '*', 'P', 'g', 'E', 'i', 'y', '2', '4', 'f', 'V', '_', '[', 'O', 'J', '9', '@', '6', '%', 'N', 'l', '3'}


In [46]:
print(len(unique_char_2))

96


In [3]:
df = pd.read_pickle('train_df.pkl')
df.to_json('train_df.json')

In [4]:
!ls

README.md                        py_df.pkl
cpp_df.pkl                       s2s.h5
ex2.cpp                          s2s2019-02-28 23:42:39.058274.h5
literature                       s2s_4_25_pm.h5
mongo2pandas.ipynb               s2s_copy.h5
mongo2pandas.py                  sb
presentation                     scraping.ipynb
py2cpp-Copy1.ipynb               scraping.py
py2cpp.ipynb                     scratch
py2cpp.py                        scratch_pad.txt
py2cpp_aws.py                    train_df.json
py2cpp_v2.ipynb                  train_df.pkl


In [6]:
df_json = pd.read_json('train_df.json')

In [9]:
df_json.sort_index()

,challenge_title,python_code,cpp_code
0,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
1,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
2,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
3,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
4,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
5,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
6,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
7,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
8,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
9,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...


In [10]:
df == df_json.sort_index()

,challenge_title,python_code,cpp_code
0,True,True,True
1,True,True,True
2,True,True,True
3,True,True,True
4,True,True,True
5,True,True,True
6,True,True,True
7,True,True,True
8,True,True,True
9,True,True,True


In [28]:
df = pd.read_pickle('train_df.pkl')

In [27]:
#print(df.python_code.iloc[0])
print(df.cpp_code.iloc[5])

#include <cmath>
#include <cstdio>
#include <vector>
#include <iostream>
#include <algorithm>
using namespace std;


int main() {
    int n, sum =0, temp;
    cin>>n;
    for(int i=0;i<n;i++)
        {
        cin >> temp;
        sum = sum +temp;
    }
    cout << sum;
    return 0;
}


In [32]:
max_code_length = 500
df=df[df['cpp_code'].map(lambda x : len(x))<max_code_length - 1]
df=df[df['python_code'].map(lambda x : len(x))<max_code_length - 1]

In [34]:
df.challenge_title.value_counts()

Simple Array Sum                    7432
A Very Big Sum                      4869
Counting Sort 1                     1779
Flipping bits                       1617
The Love-Letter Mystery             1469
Counting Sort 2                     1379
Compare the Triplets                1230
Diagonal Difference                 1115
Pangrams                             880
Breaking the Records                 853
Find the Median                      838
Plus Minus                           805
Alternating Characters               776
The Great XOR                        738
Sansa and XOR                        555
Maximizing XOR                       535
Making Anagrams                      507
Funny String                         504
Sum vs XOR                           493
AND Product                          466
Lonely Integer                       441
Game of Thrones - I                  420
Time Conversion                      413
Gemstones                            335
String Construct

In [35]:
len(df)

38936

In [36]:
df_json = pd.read_json('train_df.json')

In [37]:
df = df_json

In [38]:
df

,challenge_title,python_code,cpp_code
0,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
1,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
10,Simple Array Sum,"n = int(input())\nnums = list(map(int, input()...",#include <cmath>\n#include <cstdio>\n#include ...
100,Simple Array Sum,"n = int(input())\nprint(sum(map(int, input().s...",#include <cmath>\n#include <cstdio>\n#include ...
1000,Simple Array Sum,N = int(input())\nA = [int(value) for value in...,#include<iostream>\n\nusing namespace std;\n\n...
10000,Simple Array Sum,import math\n\nclass Adder:\n def sum(numList...,#include <cmath>\n#include <cstdio>\n#include ...
100000,Fraudulent Activity Notifications,#!/bin/python3\n\nimport sys\n\n\nDEBUG = 0\n\...,#include<bits/stdc++.h>\nusing namespace std;\...
100001,Fraudulent Activity Notifications,#!/bin/python3\n\nimport sys\n\n\nDEBUG = 0\n\...,#include <algorithm>\n#include <iostream>\n#in...
100002,Fraudulent Activity Notifications,#!/bin/python3\n\nimport sys\n\n\nDEBUG = 0\n\...,#include <iostream>\n#include <vector>\n\nusin...
100003,Fraudulent Activity Notifications,#!/bin/python3\n\nimport sys\n\n\nDEBUG = 0\n\...,#include <iostream>\n#include <vector>\n#inclu...


In [49]:
np.random.randint(100,size = (10,))

array([ 0, 97, 55, 31,  0, 35, 97, 57, 46, 64])